# Purpose of this notebook:
- Implement attribute-orthogonal multi-task training
- Train a model for engagement grading with the DAiSEE dataset

### Status:
- Multi-task training is implemented, but overfitting on the test set for gender
- attribute-orthogonal loss still needs to be implemented

# Setup
- This requires a folder structure similar to:

parent directory<br>
├─this notebook<br>
├─/dataset<br>
│  ├─/GenderClips<br>
│  │  ├─Females<br>
│  │  └─Males<br>
│  ├─/Labels<br>
│  ├─/Test<br>
│  ├─/Train<br>
│  └─/Validation<br>
├─/OUI gender dataset<br>
│  └─/OUI_model.h5

- Note: .gitignore includes the /dataset folder so that it can be co-located with the git repo for ease of use

# Sections
[section 1](#section1)<br />
[section 2](#section2)<br />
[section 3](#section3)<br />

# Imports

In [1]:
#!pip install -r requirements.txt

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model

from tqdm import tqdm

import pandas as pd  # used for storing a tabular representation of the dataset, similar to XLS files.
from pathlib import Path # used to check if the saved model files and accessories.
import requests #used to request remote judge.csv evaluation 
from sklearn.preprocessing import StandardScaler  # used for normalization of dataset
from sklearn.preprocessing   import LabelBinarizer    # used for splitting the gender column
from sklearn.preprocessing   import MinMaxScaler      # used for normalization of dataset
from sklearn.model_selection import train_test_split  # used for performing the train-test split of a dataframe
import cv2                                            # OpenCV used for image processing
import random   #random number generator
import datetime #used to get current date/time
import math     #math/numerical functions
import os       #os specific functions, like file open/close etc.
import gc       #garbage collection module -- used to manually clean up memory spaces/references.

from sklearn.preprocessing import OneHotEncoder   #My favorite categorical to numerical feature conversion tool
from tensorflow import keras  # keras used for construction of the Artificial neural network
from keras.models import Model, Sequential #keras model architectures
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D #types of layers
from keras.losses import mean_squared_error, huber, log_cosh  #built-in loss 
from tensorflow.python.keras.saving import hdf5_format  #used for saving models 
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard  #callbacks
from keras.models import model_from_json  #used for loading model architecture from json file
import h5py  #saved model type

import matplotlib.pyplot as plt  # used for training visualization
import numpy as np  # numpy arrays used for matrix computations

from keras.applications import xception
from keras import backend as K
from keras.utils import np_utils

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# File handling imports
import shutil

In [3]:
# === Extra Configurations for the GPU Environment === #
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0: #If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Discovered devices: {physical_devices}')

Discovered devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
device_spec = tf.DeviceSpec(job ="localhost", replica = 0, device_type = "CPU")

In [5]:
print('Device Spec: ', device_spec.to_string())

Device Spec:  /job:localhost/replica:0/device:CPU:*


# References:
- https://github.com/zaid478/Transfer-Learning-from-Xception-Model-in-Keras-/blob/master/transfer_learn.py

# Important Config

In [6]:
train_path = 'dataset/Train/'
test_path = 'dataset/Test/'
image_shape = (224, 299, 3) # HEIGHT, WIDTH, CHANNELS

# Train a new Xception model

In [7]:
xception_tl = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=image_shape,
    pooling=None,
    classes=4,
    classifier_activation="softmax",
)

In [8]:
# Freeze training on all layers
for layer in xception_tl.layers:
    layer.trainable = False

# multi-task model references
https://medium.com/swlh/multi-task-learning-with-tf-keras-5b28dd60246e <br>
https://datascience.stackexchange.com/questions/27498/multi-task-learning-in-keras

In [9]:
# https://keras.io/api/layers/initializers/
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

In [10]:
x = xception_tl.output
x = GlobalAveragePooling2D()(x)
x = Dropout(rate=0.50)(x)

bifurcation = Dense(
    128,
    activation='relu',
    name='bifurcation_layer',
    kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
)(x)

cls1 = Dense(64, activation='relu', name='cls1_learning_layer', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(bifurcation)
cls1b = Dense(32, activation='relu', name='cls1b_learning_layer', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(cls1)
cls1_y = Dense(4, activation='softmax', name='cls1_output', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(cls1b)

cls2 = Dense(64, activation='relu', name='cls2_learning_layer', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(bifurcation)
cls2b = Dense(32, activation='relu', name='cls2b_learning_layer', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(cls2)
cls2_y = Dense(1, activation='sigmoid', name='cls2_output', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=1.))(cls2b)

xception_tl_DAiSEE=Model(inputs=xception_tl.input, outputs=[cls1_y, cls2_y])

In [11]:
xception_tl_DAiSEE.output

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'cls1_output')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'cls2_output')>]

In [12]:
xception_tl_DAiSEE.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 37, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 19, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 19, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

# Create labels list

In [13]:
#validation_dataset_location = './dataset/Validation'

labels = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'Labels', 'AllLabels.csv'))
females = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'GenderClips', 'Females'), header=None)
males = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'GenderClips', 'Males'), header=None)

females_list = [x[0] for x in females.values.tolist()]
males_list = [x[0] for x in males.values.tolist()]

# Add gender feature columns
labels['male'] = labels.apply(lambda x: x['ClipID'] in males_list, axis=1)
labels['female'] = labels.apply(lambda x: x['ClipID'] in females_list, axis=1)

In [14]:
labels

,ClipID,Boredom,Engagement,Confusion,Frustration,male,female
0,1100011002.avi,0,2,0,0,True,False
1,1100011003.avi,0,2,0,0,True,False
2,1100011004.avi,0,3,0,0,True,False
3,1100011005.avi,0,3,0,0,True,False
4,1100011006.avi,0,3,0,0,True,False
...,...,...,...,...,...,...,...
8920,9877360164.avi,1,3,0,0,False,True
8921,9877360165.avi,0,3,0,0,False,True
8922,9877360166.avi,1,3,0,2,False,True
8923,9877360168.avi,1,3,1,1,False,True


In [15]:
testLabels = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'Labels', 'TestLabels.csv'))

In [16]:
labels['ClipID'].values.tolist()

['1100011002.avi',
 '1100011003.avi',
 '1100011004.avi',
 '1100011005.avi',
 '1100011006.avi',
 '1100011007.avi',
 '1100011008.avi',
 '1100011009.avi',
 '1100011010.avi',
 '1100011011.avi',
 '1100011012.avi',
 '1100011013.avi',
 '1100011014.avi',
 '1100011015.avi',
 '1100011016.avi',
 '1100011017.avi',
 '1100011018.avi',
 '1100011019.avi',
 '1100011020.avi',
 '1100011021.avi',
 '1100011022.avi',
 '1100011023.avi',
 '1100011025.avi',
 '1100011026.avi',
 '1100011027.avi',
 '1100011028.avi',
 '1100011029.avi',
 '1100011031.avi',
 '1100011032.avi',
 '1100011034.avi',
 '1100011035.avi',
 '1100011037.avi',
 '1100011038.avi',
 '1100011040.avi',
 '1100011046.avi',
 '1100011047.avi',
 '1100011048.avi',
 '1100011049.avi',
 '1100011050.avi',
 '1100011051.avi',
 '1100011052.avi',
 '1100011053.avi',
 '1100011054.avi',
 '1100011055.avi',
 '1100011056.avi',
 '1100011057.avi',
 '1100011058.avi',
 '1100011059.avi',
 '1100011060.avi',
 '1100011062.avi',
 '1100011063.avi',
 '1100011064.avi',
 '1100011066

In [17]:
missing = len(set(testLabels['ClipID'].values.tolist()) - set(labels['ClipID'].values.tolist()))
print(f'There are {missing} samples without labels!')

There are 61 samples without labels!


In [18]:
labels['ID_num'] = labels['ClipID'].str[:-4]

In [19]:
# we want to make sure that there is a binary constraint
assert labels.loc[labels['male']!=labels['female']].shape == labels.shape

In [20]:
# Create a single column for gender
labels['gender'] = labels.apply(lambda x: 1 if x['ClipID'] in males_list else 0, axis=1)

In [21]:
labels.columns

Index(['ClipID', 'Boredom', 'Engagement', 'Confusion', 'Frustration ', 'male',
       'female', 'ID_num', 'gender'],
      dtype='object')

In [22]:
labels.drop(['male', 'female'], axis=1, inplace=True)

In [23]:
# We're only concerned with engagement
labels.drop(['Boredom', 'Confusion', 'Frustration '], axis=1, inplace=True)

In [24]:
labels['Engagement']

0       2
1       2
2       3
3       3
4       3
       ..
8920    3
8921    3
8922    3
8923    3
8924    1
Name: Engagement, Length: 8925, dtype: int64

In [25]:
labels_ohe = pd.get_dummies(labels.Engagement)

In [26]:
labels_ohe.head()

,0,1,2,3
0,0,0,1,0
1,0,0,1,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [27]:
labels_ohe['tensor'] = labels_ohe.apply(lambda x: np.array([x[0], x[1], x[2], x[3]]), axis=1)

In [28]:
labels_ohe

,0,1,2,3,tensor
0,0,0,1,0,"[0, 0, 1, 0]"
1,0,0,1,0,"[0, 0, 1, 0]"
2,0,0,0,1,"[0, 0, 0, 1]"
3,0,0,0,1,"[0, 0, 0, 1]"
4,0,0,0,1,"[0, 0, 0, 1]"
...,...,...,...,...,...
8920,0,0,0,1,"[0, 0, 0, 1]"
8921,0,0,0,1,"[0, 0, 0, 1]"
8922,0,0,0,1,"[0, 0, 0, 1]"
8923,0,0,0,1,"[0, 0, 0, 1]"


In [29]:
labels = labels.join(labels_ohe['tensor'])

In [30]:
labels.drop(['Engagement'], axis=1, inplace=True)
labels.drop(['ClipID'], axis=1, inplace=True)

In [31]:
labels.head()

,ID_num,gender,tensor
0,1100011002,1,"[0, 0, 1, 0]"
1,1100011003,1,"[0, 0, 1, 0]"
2,1100011004,1,"[0, 0, 0, 1]"
3,1100011005,1,"[0, 0, 0, 1]"
4,1100011006,1,"[0, 0, 0, 1]"


In [32]:
print("building training set")
missing_count = 0
train_set = []
for filename in tqdm(os.listdir(train_path)):
    try:
        sample_ID = filename[:filename.index('-')]
        #engagement = labels[labels['ID_num']==sample_ID].values.tolist()[0][1:-1]
        #gender = labels[labels['ID_num']==sample_ID].values.tolist()[1]
        row = labels[labels['ID_num']==sample_ID]
        engagement = row['tensor'].values[0]
        gender = row['gender'].values[0]
        train_set.append([filename, engagement, gender])
    except IndexError:
        missing_count += 1
print(f'There are {missing_count} samples without labels!')

building training set


100%|██████████████████████████████████████████████████████████████████████████| 38374/38374 [00:31<00:00, 1200.56it/s]

There are 0 samples without labels!


In [33]:
print("building test set")
missing_count = 0
test_set = []
for filename in tqdm(os.listdir(test_path)):
    try:
        sample_ID = filename[:filename.index('-')]
        #engagement = labels[labels['ID_num']==sample_ID].values.tolist()[0][1:-1]
        #gender = labels[labels['ID_num']==sample_ID].values.tolist()[1]
        row = labels[labels['ID_num']==sample_ID]
        engagement = row['tensor'].values[0]
        gender = row['gender'].values[0]
        test_set.append([filename, engagement, gender])
    except IndexError:
        missing_count += 1
print(f'There are {missing_count} samples without labels!')

building test set


100%|██████████████████████████████████████████████████████████████████████████| 13062/13062 [00:10<00:00, 1227.53it/s]

There are 1001 samples without labels!


In [34]:
train_df = pd.DataFrame(train_set, columns=['filename', 'engagement', 'gender'])

In [35]:
test_df = pd.DataFrame(test_set, columns=['filename', 'engagement', 'gender'])

In [36]:
train_df.describe()

,gender
count,38374.000000
mean,0.682050
std,0.465686
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [37]:
train_df.head()

,filename,engagement,gender
0,1100011002-1.jpg,"[0, 0, 1, 0]",1
1,1100011002-2.jpg,"[0, 0, 1, 0]",1
2,1100011002-3.jpg,"[0, 0, 1, 0]",1
3,1100011002-4.jpg,"[0, 0, 1, 0]",1
4,1100011002-5.jpg,"[0, 0, 1, 0]",1


# ImageDataGenerator concept
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
- https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_dataframe/
- https://stackoverflow.com/questions/60621008/imagedatagenerator-for-multi-task-output-in-keras-using-flow-from-directory

In [39]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input
)
    

In [40]:
training_generator = image_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=train_path,
    x_col="filename",
    y_col=["engagement", "gender"],
    batch_size=32,
    shuffle=True,
    seed=42,
    class_mode='multi_output'
)

Found 38374 validated image filenames.


In [41]:
validation_generator = image_generator.flow_from_dataframe(
    dataframe=test_df,
    directory=test_path,
    x_col="filename",
    y_col=["engagement", "gender"],
    batch_size=32,
    shuffle=True,
    seed=42,
    class_mode='multi_output'
)

Found 12061 validated image filenames.


# Implement Attribute Orthogonal Regularized Loss

In [ ]:
def orthogonal_loss(y_true, y_pred, alpha=1):
    # requires cls1, cls2 layers to be defined through functional API
    cls1_cls2 = tf.linalg.matmul(cls1, cls2, transpose_b=True)
    numerator = tf.norm(cls1_cls2, ord=1)
    cls1_norm = tf.norm(cls1, ord=2)
    cls2_norm = tf.norm(cls2, ord=2)
    denominator = tf.math.multiply(cls1_norm, cls2_norm)
    loss_ortho = tf.math.divide_no_nan(numerator, denominator)
    
    loss_categorical = keras.losses.categorical_crossentropy(y_true, y_pred)
    
    return loss_categorical + alpha * loss_ortho

# Set up for training

In [42]:
%reload_ext tensorboard
model_path = os.path.join('saved_models/model_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '_.sav')
log_dir = os.path.join("logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

monitor_accuracy = 'val_cls2_output_accuracy'
monitor_mae = 'val_cls2_output_mean_absolute_error'

tensorboard_cbk = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_cbk = EarlyStopping(monitor=monitor_accuracy, patience=10, verbose=0, mode='min')
mcp_save_cbk = ModelCheckpoint(model_path+'.mcp.hdf5', save_best_only=True, monitor=monitor_accuracy, mode='min')
reduce_lr_plateau_cbk = ReduceLROnPlateau(monitor=monitor_mae, factor=0.1, patience=7, verbose=1, mode='min')
callbacks = [early_stopping_cbk, mcp_save_cbk, reduce_lr_plateau_cbk, tensorboard_cbk]

In [43]:
losses = {
    'cls1_output': 'categorical_crossentropy',
    'cls2_output': 'binary_crossentropy'
}
lossWeights = {"cls1_output": 1.0, "cls2_output": 5.0}

In [44]:
xception_tl_DAiSEE.compile(
    loss = losses,
    loss_weights=lossWeights,
    optimizer='adam',
    metrics = ['mean_absolute_error', 'accuracy']
)

In [45]:
history = xception_tl_DAiSEE.fit(
    training_generator,
    validation_data=validation_generator,
    batch_size=64,
    epochs=50,
    #steps_per_epoch=100, # Set temporarily for frequent validation
    callbacks=callbacks)

Epoch 1/50
1200/1200 [==============================] - 455s 374ms/step - loss: 684.9946 - cls1_output_loss: 403.2958 - cls2_output_loss: 56.3398 - cls1_output_mean_absolute_error: 0.2640 - cls1_output_accuracy: 0.4719 - cls2_output_mean_absolute_error: 0.0917 - cls2_output_accuracy: 0.9083 - val_loss: 687.0070 - val_cls1_output_loss: 79.2861 - val_cls2_output_loss: 121.5442 - val_cls1_output_mean_absolute_error: 0.2704 - val_cls1_output_accuracy: 0.4593 - val_cls2_output_mean_absolute_error: 0.2116 - val_cls2_output_accuracy: 0.7882 - lr: 0.0010
Epoch 2/50
1200/1200 [==============================] - 429s 357ms/step - loss: 93.4226 - cls1_output_loss: 61.7405 - cls2_output_loss: 6.3364 - cls1_output_mean_absolute_error: 0.2660 - cls1_output_accuracy: 0.4678 - cls2_output_mean_absolute_error: 0.0232 - cls2_output_accuracy: 0.9768 - val_loss: 515.7750 - val_cls1_output_loss: 7.9535 - val_cls2_output_loss: 101.5642 - val_cls1_output_mean_absolute_error: 0.2579 - val_cls1_output_accuracy:

# END